In [1]:
from dfttools import *
from dfttools import g
from PyMCP2221A import PyMCP2221A
mcp = PyMCP2221A.PyMCP2221A()
mcp.I2C_Init()

def i2c_write_callback( device_address: int, register_address: int, value: int, register: dict) -> bool:
    if mcp is None:
        print("I2C bridge not available")
        return False
    try:
        # read_data_bytes  = self.i2c_read_callback( device_address, register_address, register)
        # Write the register address to prepare for reading
        mcp.I2C_Write(device_address, bytearray([0x00,register_address]))
        
        # # Read one byte from the device
        read_data_bytes = mcp.I2C_Read(device_address, 1)
        if not read_data_bytes or len(read_data_bytes) < 1:
            print("Failed to read data from I2C device")
            return False
        
        read_data_int = read_data_bytes[0]  # get integer value of first byte

        mask = int(register['Mask'], 16) if 'Mask' in register else 0xFF
        lsb = int(register.get('POS', 0))  # default to 0 if POS not given

        # Prepare new value to write with masked bits updated
        # Shift value by LSB to align bits, then mask and combine
        new_value = (read_data_int & ~mask) | (value  )
        # Write back register address plus new_value to device
        # Depending on device protocol, address might need to be sent before data
        mcp.I2C_Write(device_address, bytearray([0x00,register_address, new_value]))
        mcp.I2C_Write(device_address, bytearray([0x00,register_address]))
        
        # # Read one byte from the device
        read_data = mcp.I2C_Read(device_address, 1)[-1]
        print(f"Writing 0x{new_value:02X} to device 0x{device_address:X}, register 0x{register_address:X}, post read data 0x{read_data:X}")

        return True
    except Exception as e:
        print(f"i2c_write_callback error: {e}")
        return False
# I2C read callback with optional bit masking
def i2c_read_callback( device_address: int, register_address: int, register: dict = None) -> int:
    if mcp is None:
        print("I2C bridge not available")
        return 0
    try:
        # append 0x00 for the 16bit register address 
        mcp.I2C_Write(device_address, bytearray([0x00,register_address]))
        read_data = mcp.I2C_Read(device_address, 1)
        if not read_data:
            print("I2C read returned no data")
            return 0
        raw_value = read_data[0]
        # print(raw_value)
        # if register:
        #     mask = int(register['Mask'], 16)
        #     LSB = register['POS']
        #     value = (raw_value & mask) 
        # else:
        #     value = raw_value
        print(f"Read 0x{raw_value:02X} from device 0x{device_address:X}, register 0x{register_address:X}")
        return raw_value
    except Exception as e:
        print(f"i2c_read_callback error: {e}")
        return 0
g.hardware_callbacks['i2c_write'] = i2c_write_callback
g.hardware_callbacks['i2c_read'] = i2c_read_callback
I2C_REG_WRITE(0x38,0xFE,0,0)
I2C_WRITE(device_address="0x38",field_info={'fieldname': 'island_bias_connect_time', 'length': 2, 'registers': [{'REG': '0x2B', 'POS': 2, 'RegisterName': 'ISLAND_BIAS_SETTINGS_1', 'RegisterLength': 8, 'Name': 'island_bias_connect_time[1:0]', 'Mask': '0xC', 'Length': 2, 'FieldMSB': 1, 'FieldLSB': 0, 'Attribute': '000NNNNN', 'Default': '0x02', 'User': 'YYYYYYYY', 'Clocking': 'SMB', 'Reset': 'C', 'PageName': 'PAG0'}]},write_value=0x2)
ext_offset_comp = I2C_READ(device_address="0x38",field_info={'fieldname': 'island_bias_connect_time', 'length': 2, 'registers': [{'REG': '0x2B', 'POS': 2, 'RegisterName': 'ISLAND_BIAS_SETTINGS_1', 'RegisterLength': 8, 'Name': 'island_bias_connect_time[1:0]', 'Mask': '0xC', 'Length': 2, 'FieldMSB': 1, 'FieldLSB': 0, 'Attribute': '000NNNNN', 'Default': '0x02', 'User': 'YYYYYYYY', 'Clocking': 'SMB', 'Reset': 'C', 'PageName': 'PAG0'}]},expected_value=0x1)
print(ext_offset_comp)

Writing 0x09 to device 0x38, register 0x2B, post read data 0x9
Read 0x09 from device 0x38, register 0x2B
9
1
